In [ ]:
!pip install requests beautifulsoup4 lxml pandas

In [ ]:
import sqlite3
from pathlib import Path
from typing import Dict, Any

DB_PATH = Path("/content/g1_goias_news.db")  # vai ficar salvo no /content do Colab

CREATE_TABLE_SQL = """
CREATE TABLE IF NOT EXISTS g1_goias_news (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    title TEXT NOT NULL,
    subtitle TEXT,
    url TEXT NOT NULL UNIQUE,
    section TEXT,
    city TEXT,
    author TEXT,
    published_at TEXT,
    updated_at TEXT,
    text TEXT,
    tags TEXT,
    raw_html TEXT,
    scraped_at TEXT NOT NULL
);
"""

def get_conn():
    DB_PATH.parent.mkdir(parents=True, exist_ok=True)
    conn = sqlite3.connect(DB_PATH)
    conn.execute("PRAGMA journal_mode=WAL;")
    conn.execute("PRAGMA foreign_keys = ON;")
    return conn

def init_db():
    conn = get_conn()
    with conn:
        conn.execute(CREATE_TABLE_SQL)
    conn.close()

def insert_news(conn, record: Dict[str, Any]):
    sql = """
    INSERT OR IGNORE INTO g1_goias_news
    (title, subtitle, url, section, city, author,
     published_at, updated_at, text, tags, raw_html, scraped_at)
    VALUES (:title, :subtitle, :url, :section, :city, :author,
            :published_at, :updated_at, :text, :tags, :raw_html, :scraped_at)
    """
    with conn:
        conn.execute(sql, record)


In [ ]:
from dateutil import parser as dateparser
from datetime import datetime, timedelta, timezone

def parse_datetime(text: str):
    if not text:
        return None
    try:
        dt = dateparser.parse(text)
        return dt
    except Exception:
        return None


In [ ]:
def fetch_article(url: str) -> Dict:
    print(f"[ARTICLE] Baixando: {url}")
    soup = get_soup(url)

    # Título (costuma ser <h1> mesmo)
    title_tag = soup.find("h1")
    title = title_tag.get_text(strip=True) if title_tag else None

    # Subtítulo (olhar no inspecionar: pode ser h2, div, etc.)
    subtitle_tag = soup.find("h2")
    subtitle = subtitle_tag.get_text(strip=True) if subtitle_tag else None

    # Autor (ajustar conforme o HTML real)
    author_tag = (
        soup.find(attrs={"itemprop": "author"})
        or soup.find(class_="content-author__name")
    )
    author = author_tag.get_text(strip=True) if author_tag else None

    # Datas
    published_at = None
    updated_at = None
    time_tags = soup.find_all("time")
    if time_tags:
        published_at = parse_datetime(
            time_tags[0].get("datetime") or time_tags[0].get_text()
        )
        if len(time_tags) > 1:
            updated_at = parse_datetime(
                time_tags[-1].get("datetime") or time_tags[-1].get_text()
            )

    # Seção (ajustar seletor conforme o HTML)
    section = None
    section_tag = soup.find("a", {"data-analytics": "header-section"})
    if section_tag:
        section = section_tag.get_text(strip=True)

    # Corpo da notícia
    # IMPORTANTE: veja no inspecionar qual é a div correta
    # Ex. típico: div.mc-body ou div#materia etc.
    container = soup  # troque por algo como soup.find("div", class_="mc-body")
    paragraphs_tags = container.find_all("p")
    paragraphs = [p.get_text(" ", strip=True) for p in paragraphs_tags]
    text = "\n".join(paragraphs) if paragraphs else None

    # Cidade (heurística)
    city = None
    if paragraphs:
        first_line = paragraphs[0]
        if " - " in first_line:
            maybe_city = first_line.split(" - ")[0]
            if len(maybe_city.split()) <= 4:
                city = maybe_city

    # Tags (se houver links de tag)
    tags = None
    tags_container = soup.find_all("a", rel="tag")
    if tags_container:
        tags_list = [t.get_text(strip=True) for t in tags_container]
        tags = json.dumps(tags_list, ensure_ascii=False)

    scraped_at = datetime.utcnow().isoformat()

    return {
        "title": title,
        "subtitle": subtitle,
        "url": url,
        "section": section,
        "city": city,
        "author": author,
        "published_at": published_at,
        "updated_at": updated_at,
        "text": text,
        "tags": tags,
        "raw_html": str(soup),
        "scraped_at": scraped_at,
    }

In [ ]:
import requests
from bs4 import BeautifulSoup
import time
from datetime import datetime, timedelta, timezone

# 1) Ajuste ESSA constante com base na URL que você achou no DevTools
# Exemplo (você deve trocar pelo formato REAL que viu):
FEED_URL_TEMPLATE = "https://g1.globo.com/go/goias/ultimas-noticias/index/feed/pagina-{page}.ghtml"
HEADERS = {
    "User-Agent": "Mozilla/5.0 (compatible; G1GoiasScraper/1.0; +https://seu-email-ou-site)"
}

def get_soup(url: str) -> BeautifulSoup:
    resp = requests.get(url, headers=HEADERS, timeout=10)
    resp.raise_for_status()
    return BeautifulSoup(resp.text, "lxml")


def fetch_listing_links_from_feed(page: int) -> list[str]:
    """
    Usa a mesma URL que o botão 'veja mais' chamaria,
    e devolve os links de notícias encontrados nessa 'página' do feed.
    """
    url = FEED_URL_TEMPLATE.format(page=page)
    print(f"[FEED] Baixando página de feed: {url}")
    soup = get_soup(url)

    urls = set()

    # Aqui dá pra ser mais específico se você souber o seletor dos cards.
    # Enquanto isso, filtramos todos <a> com /go/goias/noticia/ na URL:
    for a in soup.select("a"):
        href = a.get("href")
        if not href:
            continue
        if "/go/goias/noticia/" in href and href.startswith("https://"):
            urls.add(href)

    return sorted(urls)


def crawl_ultimas_noticias_ate_6_meses(max_pages: int = 300, months: int = 6):
    """
    Simula 'scroll + veja mais' usando o endpoint de feed/pagina-{n},
    e vai salvando notícias até achar coisas mais velhas que `months` meses.

    max_pages: limite de páginas de feed pra não cair em loop infinito
    """
    from __main__ import init_db, get_conn, insert_news, fetch_article, parse_datetime

    init_db()
    conn = get_conn()

    cutoff = datetime.now(timezone.utc) - timedelta(days=30 * months)
    print(f"[INFO] Data de corte (aprox.): {cutoff}")

    seen_urls = set()
    pages_so_far = 0
    pages_only_old = 0  # contagem de páginas em que tudo é velho

    for page in range(1, max_pages + 1):
        pages_so_far += 1
        try:
            article_urls = fetch_listing_links_from_feed(page)
        except Exception as e:
            print(f"[STOP] Erro ao baixar feed da página {page}: {e}")
            break

        if not article_urls:
            print(f"[STOP] Feed da página {page} veio vazio. Encerrando.")
            break

        page_has_newer = False

        for url in article_urls:
            if url in seen_urls:
                continue
            seen_urls.add(url)

            try:
                record = fetch_article(url)
            except Exception as e:
                print(f"[ERROR] Falha ao baixar artigo {url}: {e}")
                continue

            pub_dt_raw = record.get("published_at")
            pub_dt = parse_datetime(pub_dt_raw) if pub_dt_raw else None

            pub_dt_utc = None
            if pub_dt is not None:
                if pub_dt.tzinfo is None:
                    pub_dt_utc = pub_dt.replace(tzinfo=timezone.utc)
                else:
                    pub_dt_utc = pub_dt.astimezone(timezone.utc)

            # Se temos data e ela é mais velha do que a data de corte:
            if pub_dt_utc is not None and pub_dt_utc < cutoff:
                print(
                    f"[SKIP] {record['title'][:60]}... ({pub_dt_utc.date()}) "
                    f"< corte ({cutoff.date()})"
                )
                continue

            # Se não temos data, eu prefiro salvar mesmo assim (você pode mudar isso)
            insert_news(conn, record)
            page_has_newer = True
            print(f"[OK] salvo: {record['title'][:80]}")

            time.sleep(1)

        if not page_has_newer:
            pages_only_old += 1
            # se algumas páginas seguidas só têm notícia velha, podemos parar
            if pages_only_old >= 3:
                print(
                    f"[STOP] {pages_only_old} páginas seguidas só com notícia mais velha que o corte. Encerrando."
                )
                break
        else:
            pages_only_old = 0

        # descanso entre pages de feed
        time.sleep(1)

    conn.close()
    print(f"[DONE] Processadas {pages_so_far} páginas de feed.")


In [ ]:
for p in range(1, 4):
    print(FEED_URL_TEMPLATE.format(page=p))


https://g1.globo.com/go/goias/ultimas-noticias/index/feed/pagina-1.ghtml
https://g1.globo.com/go/goias/ultimas-noticias/index/feed/pagina-2.ghtml
https://g1.globo.com/go/goias/ultimas-noticias/index/feed/pagina-3.ghtml


In [ ]:
crawl_ultimas_noticias_ate_6_meses(max_pages=28, months=6)

[INFO] Data de corte (aprox.): 2025-06-13 20:16:24.565987+00:00
[FEED] Baixando página de feed: https://g1.globo.com/go/goias/ultimas-noticias/index/feed/pagina-1.ghtml
[ARTICLE] Baixando: https://g1.globo.com/go/goias/noticia/2025/12/10/cantor-sertanejo-e-roubado-em-parque-de-goiania-video.ghtml


/tmp/ipython-input-253435095.py:63: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  scraped_at = datetime.utcnow().isoformat()
/tmp/ipython-input-2258786332.py:47: DeprecationWarning: The default datetime adapter is deprecated as of Python 3.12; see the sqlite3 documentation for suggested replacement recipes
  conn.execute(sql, record)


[OK] salvo: Cantor sertanejo é roubado em parque de Goiânia; vídeo
[ARTICLE] Baixando: https://g1.globo.com/go/goias/noticia/2025/12/10/caso-valerio-luiz-stj-mantem-multa-a-advogados-que-abandonaram-juri.ghtml
[OK] salvo: Caso Valério Luiz: STJ mantém multa a advogados que abandonaram júri
[ARTICLE] Baixando: https://g1.globo.com/go/goias/noticia/2025/12/10/corpo-de-jovem-que-desapareceu-apos-nadar-em-rio-e-encontrado.ghtml
[OK] salvo: Corpo de jovem que desapareceu após nadar em rio é encontrado
[ARTICLE] Baixando: https://g1.globo.com/go/goias/noticia/2025/12/10/crianca-de-10-anos-que-foi-esfaqueada-multiplas-vezes-estava-em-cima-de-pe-de-manga-antes-de-ser-atacada-diz-familia.ghtml
[OK] salvo: Menino de 10 anos que foi esfaqueado múltiplas vezes estava em cima de pé de man
[ARTICLE] Baixando: https://g1.globo.com/go/goias/noticia/2025/12/10/ex-presidente-do-sindicato-rural-de-rio-verde-e-denunciado-por-crimes-sexuais-contra-funcionarias.ghtml
[OK] salvo: Ex-presidente do Sindicato R

In [ ]:
import pandas as pd

conn = get_conn()
df = pd.read_sql_query("SELECT * FROM g1_goias_news", conn)
conn.close()

print(f"DataFrame criado com {len(df)} notícias.")
display(df.head())

DataFrame criado com 204 notícias.


,id,title,subtitle,url,section,city,author,published_at,updated_at,text,tags,raw_html,scraped_at
0,1,Cantor sertanejo é roubado em parque de Goiâni...,"Segundo João Victor, uma corrente de ouro foi ...",https://g1.globo.com/go/goias/noticia/2025/12/...,None,None,,2025-12-10 13:06:34.597000-03:00,2025-12-10 13:30:15.949000-03:00,"Por Letícia Fiuza , g1 Goiás\n10/12/2025 13h06...",None,"<!DOCTYPE HTML>\n<html class=""bs-channel-deskt...",2025-12-10T20:16:26.954073
1,2,Caso Valério Luiz: STJ mantém multa a advogado...,Tribunal entendeu que lei que proíbe multa a d...,https://g1.globo.com/go/goias/noticia/2025/12/...,None,None,,2025-12-10 13:12:02.461000-03:00,2025-12-10 13:12:36.910000-03:00,"Por Addan Vieira , g1 Goiás\n10/12/2025 13h12 ...",None,"<!DOCTYPE HTML>\n<html class=""bs-channel-deskt...",2025-12-10T20:16:29.060369
2,3,Corpo de jovem que desapareceu após nadar em r...,Rafael Campos havia entrado no Rio Claro para ...,https://g1.globo.com/go/goias/noticia/2025/12/...,None,None,,2025-12-10 13:58:56.218000-03:00,2025-12-10 14:10:38.305000-03:00,"Por Rafaella Barros, g1 Goiás\n10/12/2025 13h5...",None,"<!DOCTYPE HTML>\n<html class=""bs-channel-deskt...",2025-12-10T20:16:31.200642
3,4,Menino de 10 anos que foi esfaqueado múltiplas...,"Segundo a Polícia Civil, o suspeito de ter gol...",https://g1.globo.com/go/goias/noticia/2025/12/...,None,None,,2025-12-10 11:00:23.139000-03:00,2025-12-10 11:50:03.234000-03:00,"Por Yanca Cristina , g1 Goiás\n10/12/2025 11h0...",None,"<!DOCTYPE HTML>\n<html class=""bs-channel-deskt...",2025-12-10T20:16:33.514851
4,5,Ex-presidente do Sindicato Rural de Rio Verde ...,Olávio Teles Fonseca foi denunciado por estupr...,https://g1.globo.com/go/goias/noticia/2025/12/...,None,None,,2025-12-10 14:02:27.903000-03:00,2025-12-10 14:20:36.273000-03:00,"Por Yanca Cristina , Honório Jacometto, g1 Goi...",None,"<!DOCTYPE HTML>\n<html class=""bs-channel-deskt...",2025-12-10T20:16:35.774122


In [ ]:
df = df.drop(columns=["raw_html"])

In [ ]:
display(df.info(memory_usage='deep'))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 204 entries, 0 to 203
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   id            204 non-null    int64 
 1   title         204 non-null    object
 2   subtitle      204 non-null    object
 3   url           204 non-null    object
 4   section       0 non-null      object
 5   city          0 non-null      object
 6   author        204 non-null    object
 7   published_at  204 non-null    object
 8   updated_at    204 non-null    object
 9   text          204 non-null    object
 10  tags          0 non-null      object
 11  scraped_at    204 non-null    object
dtypes: int64(1), object(11)
memory usage: 3.3 MB


None

In [ ]:
df.to_csv('dataset_noticias_g1.csv')